<a href="https://colab.research.google.com/github/LeoDinga/DL_Project/blob/main/Project_DL_3_Diana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --filter=blob:none --no-checkout https://github.com/THETIS-dataset/dataset.git
%cd dataset
!git sparse-checkout init --cone
!git sparse-checkout set VIDEO_Skelet3D
!git checkout

Cloning into 'dataset'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 113 (delta 0), reused 2 (delta 0), pack-reused 110 (from 1)
Receiving objects: 100% (113/113), 214.90 KiB | 8.60 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/dataset
remote: Enumerating objects: 1217, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1217 (delta 0), reused 0 (delta 0), pack-reused 1216 (from 1)
Receiving objects: 100% (1217/1217), 507.08 MiB | 16.54 MiB/s, done.
Updating files: 100% (1218/1218), done.
Your branch is up to date with 'origin/main'.


In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is inco

In [3]:
%pip install --upgrade pip
%pip install mediapipe opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 168.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [mediapipe]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which i

In [4]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import torchvision.models as models
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score
import shutil
import random
import mediapipe as mp

In [5]:
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: False
Device name: No GPU


In [6]:
import os
print(os.listdir('VIDEO_Skelet3D'))

['smash', 'forehand_openstands', 'kick_service', 'slice_service', 'backhand', 'backhand_slice', 'flat_service', 'backhand_volley', 'forehand_volley', 'forehand_slice', 'forehand_flat', 'backhand2hands']


In [7]:
#shows the videos in each folder
data_path = 'VIDEO_Skelet3D'

# Print out the directory structure
for root, dirs, files in os.walk(data_path):
    print(f"Root: {root}")
    print(f"Dirs: {dirs}")
    print(f"Files: {files}")
    print("-" * 40)

Root: VIDEO_Skelet3D
Dirs: ['smash', 'forehand_openstands', 'kick_service', 'slice_service', 'backhand', 'backhand_slice', 'flat_service', 'backhand_volley', 'forehand_volley', 'forehand_slice', 'forehand_flat', 'backhand2hands']
Files: []
----------------------------------------
Root: VIDEO_Skelet3D/smash
Dirs: []
Files: ['p33_smash_skelet3D_s2.avi', 'p50_smash_skelet3D_s2.avi', 'p13_smash_skelet3D_s2.avi', 'p38_smash_skelet3D_s2.avi', 'p20_smash_skelet3D_s3.avi', 'p34_smash_skelet3D_s3.avi', 'p51_smash_skelet3D_s2.avi', 'p16_smash_skelet3D_s1.avi', 'p35_smash_skelet3D_s3.avi', 'p42_smash_skelet3D_s3.avi', 'p3_smash_skelet3D_s3.avi', 'p42_smash_skelet3D_s2.avi', 'p5_smash_skelet3D_s2.avi', 'p25_smash_skelet3D_s1.avi', 'p27_smash_skelet3D_s3.avi', 'p19_smash_skelet3D_s3.avi', 'p23_smash_skelet3D_s3.avi', 'p50_smash_skelet3D_s3.avi', 'p54_smash_skelet3D_s2.avi', 'p23_smash_skelet3D_s2.avi', 'p16_smash_skelet3D_s2.avi', 'p36_smash_skelet3D_s3.avi', 'p17_smash_skelet3D_s2.avi', 'p42_smash

In [8]:
def convert_video_to_npy(video_path, resize_shape=(224, 224)):
    """
    Converts a video to a numpy array of resized frames.

    Parameters:
    - video_path: The path to the video file.
    - resize_shape: The desired frame size (default is 224x224).

    Returns:
    - frames_array: Numpy array containing all frames.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []

    if not cap.isOpened():
        raise ValueError(f"Error opening video file: {video_path}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to the desired shape
        frame_resized = cv2.resize(frame, resize_shape)
        frames.append(frame_resized)

    cap.release()

    # Convert list of frames into a numpy array
    frames_array = np.array(frames)
    return frames_array

def create_subfolders(pastas_dir, output_dir):
    """
    Creates subfolders in the output directory and processes .avi files.

    Parameters:
    - pastas_dir: The root directory containing class folders.
    - output_dir: The directory where the .npy files will be saved.
    """
    for pastas_filename in os.listdir(pastas_dir):
        origem_path = os.path.join(pastas_dir, pastas_filename)

        # Check if it's a directory (ignore files)
        if os.path.isdir(origem_path):
            nova_pasta_path = os.path.join(output_dir, pastas_filename)
            os.makedirs(nova_pasta_path, exist_ok=True)

            # Process each video file in the subfolder
            videos_dir = os.path.join(pastas_dir, pastas_filename)
            for video_filename in os.listdir(videos_dir):
                if video_filename.endswith(".avi"):  # Process only .avi files
                    video_path = os.path.join(videos_dir, video_filename)
                    video_name = os.path.splitext(video_filename)[0]

                    try:
                        # Convert the video to numpy array
                        frames_array = convert_video_to_npy(video_path)

                        # Save the numpy array to a .npy file
                        output_filename = os.path.join(nova_pasta_path, f"{video_name}.npy")
                        np.save(output_filename, frames_array)
                    except ValueError as e:
                        print(f"Error processing {video_filename}: {e}")
                    except Exception as e:
                        print(f"Unexpected error with {video_filename}: {e}")

# Example usage
pastas_dir = 'VIDEO_Skelet3D'  # Directory with .avi videos
npy_dir = 'npy_videos'  # Directory to save .npy files
os.makedirs(npy_dir, exist_ok=True)

# Process videos and maintain class folders
create_subfolders(pastas_dir, npy_dir)

In [9]:
# Pad/truncate to 120 frames
def pad_or_truncate_keypoints(keypoints, target_length=120):
    """
    Pads or truncates a keypoints array to a fixed number of frames (target_length).
    If padding is needed, uses zeros.
    """
    current_length = keypoints.shape[0]

    if current_length == target_length:
        return keypoints
    elif current_length > target_length:
        return keypoints[:target_length]  # Truncate
    else:
        # Pad with zeros
        padding = np.zeros((target_length - current_length, keypoints.shape[1], keypoints.shape[2]))
        return np.concatenate([keypoints, padding], axis=0)

In [ ]:
# In theory this will extract the keypoints AND pad/truncate
from tqdm import tqdm

root_dir = "npy_videos"

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

all_keypoints = {}

# Top-level loop with a single progress bar
action_folders = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
for action_folder in tqdm(action_folders, desc="Processing actions"):
    action_path = os.path.join(root_dir, action_folder)
    all_keypoints[action_folder] = {}

    video_files = sorted([f for f in os.listdir(action_path) if f.endswith(".npy")])
    for video_file in video_files:  # No tqdm here
        video_path = os.path.join(action_path, video_file)
        try:
            sample = np.load(video_path)
            video_keypoints = []

            for frame in sample:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(frame_rgb)

                if results.pose_landmarks:
                    keypoints = [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]
                else:
                    keypoints = np.zeros((33, 3)).tolist()

                video_keypoints.append(keypoints)

            video_keypoints = np.array(video_keypoints)
            video_keypoints = pad_or_truncate_keypoints(video_keypoints, target_length=120)
            all_keypoints[action_folder][video_file] = video_keypoints

        except Exception as e:
            pass  # Suppress per-video error printing for a clean log

# Final summary
print("\n Summary of extracted keypoints:")
for action, videos in all_keypoints.items():
    print(f"- {action}: {len(videos)} videos processed.")

Processing actions:  25%|██▌       | 3/12 [10:01<29:42, 198.03s/it]